# Census - SexByAge

#### This notebook contains code for gathering Census data tables and converting them to dataframes. 
#### Please use caution when altering this code.

In [2]:
import pandas as pd
from census import Census #<-- Python wrapper for census API
import requests
import os
import matplotlib.pyplot as plt
import matplotlib

# Census API Key
from config import api_key

# provide the api key and the year to establish a session
c = Census(api_key, year=2018)

# Set an option to allow up to 300 characters to print in each column
pd.set_option('max_colwidth', 300)

In [3]:
tables = c.acs5.tables()

# The tables variable contains a list of dicts, so we can convert directly to a dataframe
table_df = pd.DataFrame(tables)

##### The cell below gathering columns names from the Census Total Population table that only contain estimate and are int data types.
##### Those table names are added to a string variable and then pulled from the census data.
##### Dictionary key names are replaced withe more meaningful names.
##### Data is saved to a dataframe and exported as a csv file.

In [6]:
table_id = 'B27001'      

# Capture the variables URL from the table_df
url = table_df.loc[table_df['name']==table_id, 'variables'].values[0]

# Make the API call
response = requests.get(url).json()

# convert the response to a DataFrame
variables = pd.DataFrame(response['variables']).transpose()

print(f"Number of available variables: {len(variables)}")

table_info = variables[(variables['predicateType']=='int') & (variables['label'].str.contains("Estimate"))]   

table_info

# table_columns = ""
# for ind in table_info.index:       
#     table_columns = table_columns + f"{ind},"
    
    
# column_list = table_columns[:-1]
# census_data = c.acs5.get(("NAME", column_list), 
#                           {'for': 'county:*'})

# for i in range(len(table_info)) : 
#     for d in census_data:
#         d[table_info.iloc[i, 0]] = d.pop(table_info.index[i]) 
    

# census_B27001_HealthInsSexByAge_df = pd.DataFrame(census_data)
# census_B27001_HealthInsSexByAge_df.to_csv("census_B27001_HealthInsSexByAge.csv", encoding="utf-8", index=False )
# census_B27001_HealthInsSexByAge_df

Number of available variables: 228


,label,predicateType,group,limit,predicateOnly,concept
B27001_031E,Estimate!!Total!!Female!!Under 6 years,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_032E,Estimate!!Total!!Female!!Under 6 years!!With health insurance coverage,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_030E,Estimate!!Total!!Female,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_023E,Estimate!!Total!!Male!!55 to 64 years!!No health insurance coverage,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_024E,Estimate!!Total!!Male!!65 to 74 years,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_021E,Estimate!!Total!!Male!!55 to 64 years,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_022E,Estimate!!Total!!Male!!55 to 64 years!!With health insurance coverage,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_027E,Estimate!!Total!!Male!!75 years and over,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_028E,Estimate!!Total!!Male!!75 years and over!!With health insurance coverage,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE
B27001_025E,Estimate!!Total!!Male!!65 to 74 years!!With health insurance coverage,int,B27001,0,True,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE


### Census Sex By Age Column Changes

In [15]:
census_B01001_SexByAge_df.dtypes

census_B01001_SexByAge_df['COUNTYFP'] = census_B01001_SexByAge_df['state'] + census_B01001_SexByAge_df['county']

census_SexByAge_sorted = census_B01001_SexByAge_df.sort_values(by=['COUNTYFP'])
census_SexByAge_sorted = census_SexByAge_sorted.reset_index(drop=True)

census_SexByAge_sorted['State Name'] = ''
census_SexByAge_sorted['County Name'] = ''


for index, row in census_SexByAge_sorted.iterrows():
    name_all = row['NAME']
    name_list = name_all.split(',')
    clean_state = name_list[1][1:]
    census_SexByAge_sorted.loc[index, 'State Name']= clean_state
    census_SexByAge_sorted.loc[index, 'County Name']= name_list[0]
    
census_SexByAge_sorted.head()

,NAME,state,county,Estimate!!Total!!Male!!30 to 34 years,Estimate!!Total!!Male!!25 to 29 years,Estimate!!Total!!Male!!40 to 44 years,Estimate!!Total!!Male!!35 to 39 years,Estimate!!Total!!Male!!50 to 54 years,Estimate!!Total!!Male!!45 to 49 years,Estimate!!Total!!Male!!62 to 64 years,...,Estimate!!Total!!Male!!Under 5 years,Estimate!!Total!!Male!!15 to 17 years,Estimate!!Total!!Male!!18 and 19 years,Estimate!!Total!!Male!!10 to 14 years,Estimate!!Total!!Male!!21 years,Estimate!!Total!!Male!!20 years,Estimate!!Total!!Male!!22 to 24 years,COUNTYFP,State Name,County Name
0,"Autauga County, Alabama",01,001,1697.0,1705.0,1576.0,1992.0,1937.0,1899.0,660.0,...,1789.0,1256.0,760.0,1754.0,396.0,399.0,910.0,01001,Alabama,Autauga County
1,"Baldwin County, Alabama",01,003,5704.0,5874.0,6233.0,6092.0,7200.0,6555.0,4224.0,...,5855.0,4134.0,2379.0,7544.0,918.0,1079.0,3444.0,01003,Alabama,Baldwin County
2,"Barbour County, Alabama",01,005,1115.0,1162.0,903.0,966.0,916.0,957.0,488.0,...,717.0,474.0,266.0,912.0,181.0,284.0,501.0,01005,Alabama,Barbour County
3,"Bibb County, Alabama",01,007,995.0,899.0,818.0,836.0,954.0,933.0,330.0,...,692.0,541.0,444.0,690.0,58.0,116.0,519.0,01007,Alabama,Bibb County
4,"Blount County, Alabama",01,009,1675.0,1669.0,2073.0,1511.0,2046.0,1955.0,1023.0,...,1813.0,1215.0,783.0,2118.0,401.0,167.0,1089.0,01009,Alabama,Blount County


#### Combine male columns into larger groups

In [16]:
census_SexByAge_sorted['Estimate!!Total!!Male!!17 and under years'] = census_SexByAge_sorted['Estimate!!Total!!Male!!Under 5 years'] + \
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!5 to 9 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!10 to 14 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!15 to 17 years']

census_SexByAge_sorted['Estimate!!Total!!Male!!18 to 44 years'] = census_SexByAge_sorted['Estimate!!Total!!Male!!18 and 19 years'] + \
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!20 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!21 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!22 to 24 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!25 to 29 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!30 to 34 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!35 to 39 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!40 to 44 years']

census_SexByAge_sorted['Estimate!!Total!!Male!!45 to 64 years'] = census_SexByAge_sorted['Estimate!!Total!!Male!!45 to 49 years'] + \
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!50 to 54 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!55 to 59 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!60 and 61 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!62 to 64 years'] 

census_SexByAge_sorted['Estimate!!Total!!Male!!65 and over years'] = census_SexByAge_sorted['Estimate!!Total!!Male!!65 and 66 years'] + \
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!67 to 69 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!70 to 74 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!75 to 79 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!80 to 84 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Male!!85 years and over']



census_SexByAge_sorted.head()

,NAME,state,county,Estimate!!Total!!Male!!30 to 34 years,Estimate!!Total!!Male!!25 to 29 years,Estimate!!Total!!Male!!40 to 44 years,Estimate!!Total!!Male!!35 to 39 years,Estimate!!Total!!Male!!50 to 54 years,Estimate!!Total!!Male!!45 to 49 years,Estimate!!Total!!Male!!62 to 64 years,...,Estimate!!Total!!Male!!21 years,Estimate!!Total!!Male!!20 years,Estimate!!Total!!Male!!22 to 24 years,COUNTYFP,State Name,County Name,Estimate!!Total!!Male!!17 and under years,Estimate!!Total!!Male!!18 to 44 years,Estimate!!Total!!Male!!45 to 64 years,Estimate!!Total!!Male!!65 and over years
0,"Autauga County, Alabama",01,001,1697.0,1705.0,1576.0,1992.0,1937.0,1899.0,660.0,...,396.0,399.0,910.0,01001,Alabama,Autauga County,6820.0,9435.0,7131.0,3488.0
1,"Baldwin County, Alabama",01,003,5704.0,5874.0,6233.0,6092.0,7200.0,6555.0,4224.0,...,918.0,1079.0,3444.0,01003,Alabama,Baldwin County,23084.0,31723.0,27511.0,18870.0
2,"Barbour County, Alabama",01,005,1115.0,1162.0,903.0,966.0,916.0,957.0,488.0,...,181.0,284.0,501.0,01005,Alabama,Barbour County,2774.0,5378.0,3541.0,2004.0
3,"Bibb County, Alabama",01,007,995.0,899.0,818.0,836.0,954.0,933.0,330.0,...,58.0,116.0,519.0,01007,Alabama,Bibb County,2574.0,4685.0,3260.0,1633.0
4,"Blount County, Alabama",01,009,1675.0,1669.0,2073.0,1511.0,2046.0,1955.0,1023.0,...,401.0,167.0,1089.0,01009,Alabama,Blount County,6839.0,9368.0,7687.0,4540.0


#### Combine female columns into larger groups

In [18]:
census_SexByAge_sorted['Estimate!!Total!!Female!!17 and under years'] = census_SexByAge_sorted['Estimate!!Total!!Female!!Under 5 years'] + \
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!5 to 9 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!10 to 14 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!15 to 17 years']

census_SexByAge_sorted['Estimate!!Total!!Female!!18 to 44 years'] = census_SexByAge_sorted['Estimate!!Total!!Female!!18 and 19 years'] + \
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!20 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!21 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!22 to 24 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!25 to 29 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!30 to 34 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!35 to 39 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!40 to 44 years']

census_SexByAge_sorted['Estimate!!Total!!Female!!45 to 64 years'] = census_SexByAge_sorted['Estimate!!Total!!Female!!45 to 49 years'] + \
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!50 to 54 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!55 to 59 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!60 and 61 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!62 to 64 years'] 

census_SexByAge_sorted['Estimate!!Total!!Female!!65 and over years'] = census_SexByAge_sorted['Estimate!!Total!!Female!!65 and 66 years'] + \
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!67 to 69 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!70 to 74 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!75 to 79 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!80 to 84 years'] +\
                                                                census_SexByAge_sorted['Estimate!!Total!!Female!!85 years and over']

census_SexByAge_sorted.head()

,NAME,state,county,Estimate!!Total!!Male!!30 to 34 years,Estimate!!Total!!Male!!25 to 29 years,Estimate!!Total!!Male!!40 to 44 years,Estimate!!Total!!Male!!35 to 39 years,Estimate!!Total!!Male!!50 to 54 years,Estimate!!Total!!Male!!45 to 49 years,Estimate!!Total!!Male!!62 to 64 years,...,State Name,County Name,Estimate!!Total!!Male!!17 and under years,Estimate!!Total!!Male!!18 to 44 years,Estimate!!Total!!Male!!45 to 64 years,Estimate!!Total!!Male!!65 and over years,Estimate!!Total!!Female!!17 and under years,Estimate!!Total!!Female!!18 to 44 years,Estimate!!Total!!Female!!45 to 64 years,Estimate!!Total!!Female!!65 and over years
0,"Autauga County, Alabama",01,001,1697.0,1705.0,1576.0,1992.0,1937.0,1899.0,660.0,...,Alabama,Autauga County,6820.0,9435.0,7131.0,3488.0,6549.0,9668.0,7547.0,4562.0
1,"Baldwin County, Alabama",01,003,5704.0,5874.0,6233.0,6092.0,7200.0,6555.0,4224.0,...,Alabama,Baldwin County,23084.0,31723.0,27511.0,18870.0,22593.0,32421.0,30110.0,21795.0
2,"Barbour County, Alabama",01,005,1115.0,1162.0,903.0,966.0,916.0,957.0,488.0,...,Alabama,Barbour County,2774.0,5378.0,3541.0,2004.0,2662.0,3574.0,3219.0,2630.0
3,"Bibb County, Alabama",01,007,995.0,899.0,818.0,836.0,954.0,933.0,330.0,...,Alabama,Bibb County,2574.0,4685.0,3260.0,1633.0,2085.0,3373.0,2889.0,2028.0
4,"Blount County, Alabama",01,009,1675.0,1669.0,2073.0,1511.0,2046.0,1955.0,1023.0,...,Alabama,Blount County,6839.0,9368.0,7687.0,4540.0,6629.0,9121.0,7768.0,5693.0


In [20]:
census_SexByAge_Combined = census_SexByAge_sorted[['NAME',
                                                'state',
                                                'county',
                                                'State Name',
                                                'County Name',
                                                'Estimate!!Total!!Male!!17 and under years',
                                                'Estimate!!Total!!Male!!18 to 44 years',
                                                'Estimate!!Total!!Male!!45 to 64 years',
                                                'Estimate!!Total!!Male!!65 and over years',
                                                'Estimate!!Total!!Male',
                                                'Estimate!!Total!!Female!!17 and under years',
                                                'Estimate!!Total!!Female!!18 to 44 years',
                                                'Estimate!!Total!!Female!!45 to 64 years',
                                                'Estimate!!Total!!Female!!65 and over years',
                                                'Estimate!!Total!!Female'
                                            ]]

census_SexByAge_Combined.head()


,NAME,state,county,State Name,County Name,Estimate!!Total!!Male!!17 and under years,Estimate!!Total!!Male!!18 to 44 years,Estimate!!Total!!Male!!45 to 64 years,Estimate!!Total!!Male!!65 and over years,Estimate!!Total!!Male,Estimate!!Total!!Female!!17 and under years,Estimate!!Total!!Female!!18 to 44 years,Estimate!!Total!!Female!!45 to 64 years,Estimate!!Total!!Female!!65 and over years,Estimate!!Total!!Female
0,"Autauga County, Alabama",01,001,Alabama,Autauga County,6820.0,9435.0,7131.0,3488.0,26874.0,6549.0,9668.0,7547.0,4562.0,28326.0
1,"Baldwin County, Alabama",01,003,Alabama,Baldwin County,23084.0,31723.0,27511.0,18870.0,101188.0,22593.0,32421.0,30110.0,21795.0,106919.0
2,"Barbour County, Alabama",01,005,Alabama,Barbour County,2774.0,5378.0,3541.0,2004.0,13697.0,2662.0,3574.0,3219.0,2630.0,12085.0
3,"Bibb County, Alabama",01,007,Alabama,Bibb County,2574.0,4685.0,3260.0,1633.0,12152.0,2085.0,3373.0,2889.0,2028.0,10375.0
4,"Blount County, Alabama",01,009,Alabama,Blount County,6839.0,9368.0,7687.0,4540.0,28434.0,6629.0,9121.0,7768.0,5693.0,29211.0
